In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [17]:
l_in = lasagne.layers.InputLayer((1,50))

In [18]:
l_hidden = lasagne.layers.DenseLayer(l_in, num_units=10, name='dens1')

In [19]:
l_out = lasagne.layers.DenseLayer(l_hidden, num_units=1, name='dens2')

In [20]:
out = lasagne.layers.get_output(l_out)

In [21]:
print out

Elemwise{mul,no_inplace}.0


In [22]:
target = T.ivector('y')
loss = lasagne.objectives.binary_crossentropy(out, target).mean()

In [23]:
all_params = lasagne.layers.get_all_params(l_out)

In [24]:
update = lasagne.updates.adagrad(loss, all_params, .01, .9)

In [25]:
train = theano.function(
    [l_in.input_var, target], loss, updates=update,
    allow_input_downcast=True,
    mode='DebugMode',
)

In [26]:
fun_loss = theano.function([l_in.input_var, target],loss,mode='DebugMode')
#fun_out = theano.function([l_in.input_var], out, mode='DebugMode', on_unused_input='warn')

In [27]:
target_v = (np.random.randint(0,2,1) * 2 - 1).astype('int32')

In [28]:
target_t = np.random.randn(1, 50)

In [29]:
fun_loss(target_t, target_v)

InvalidValueError: InvalidValueError
        type(variable) = TensorType(float64, matrix)
        variable       = Elemwise{Composite{((i0 * log(Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3))) + (i4 * log((i5 - Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3)))))}}[(0, 2)].0
        type(value)    = <type 'numpy.ndarray'>
        dtype(value)   = float64
        shape(value)   = (1, 1)
        value          = [[ inf]]
        min(value)     = inf
        max(value)     = inf
        isfinite       = False
        client_node    = None
        hint           = perform output
        specific_hint  = non-finite elements not allowed
        context        = ...
  Elemwise{Composite{((i0 * log(Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3))) + (i4 * log((i5 - Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3)))))}}[(0, 2)] [@A] ''   
   |InplaceDimShuffle{x,0} [@B] ''   
   | |y [@C]
   |TensorConstant{(1, 1) of 0.5} [@D]
   |Dot22 [@E] ''   
   | |Elemw

In [30]:
theano.printing.debugprint(fun_loss)

Elemwise{Composite{(((-i0) / i1) / i2)}}[(0, 0)] [@A] ''   15
 |Sum{acc_dtype=float64} [@B] ''   14
 | |Elemwise{Composite{((i0 * log(Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3))) + (i4 * log((i5 - Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}(i1, i2, i3)))))}}[(0, 2)] [@C] ''   13
 |   |InplaceDimShuffle{x,0} [@D] ''   1
 |   | |y [@E]
 |   |TensorConstant{(1, 1) of 0.5} [@F]
 |   |Dot22 [@G] ''   10
 |   | |Elemwise{Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}}[(0, 1)] [@H] ''   8
 |   | | |TensorConstant{(1, 1) of 0.5} [@F]
 |   | | |Dot22 [@I] ''   5
 |   | | | |input [@J]
 |   | | | |dens1.W [@K]
 |   | | |InplaceDimShuffle{x,0} [@L] ''   4
 |   | |   |dens1.b [@M]
 |   | |dens2.W [@N]
 |   |InplaceDimShuffle{x,0} [@O] ''   3
 |   | |dens2.b [@P]
 |   |Elemwise{sub,no_inplace} [@Q] ''   6
 |   | |TensorConstant{(1, 1) of 1.0} [@R]
 |   | |InplaceDimShuffle{x,0} [@D] ''   1
 |   |TensorConstant{(1, 1) of 1.0} [@R]
 |Subtensor{int64} [@S] ''   12
 | |Elemwise{Cast{float6

In [ ]:
target_t.shape

In [ ]:
train(target_t, target_v)

In [ ]:
eval_v = {l_in.input_var: target_t, target: target_v}
l_out.input_shape

In [ ]:
[(a.shape.eval(), a) for a in all_params]

In [ ]:
target_v.shape, target_t.shape

In [ ]:
target_t.dot(all_params[0].get_value()).dot(all_params[2].get_value()) > 0

In [ ]:
theano.printing.debugprint(train)

In [ ]:
theano.config.DebugMode.